<a href="https://colab.research.google.com/github/sfnesbit/CSC466-Team13/blob/main/CSC_466_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = "lime"> CSC 466 Team 13 Food Image Classification</font>
<font color = "lightgreen"> Ty Farris | Marine Cossoul | Sean Nesbit </font>

[Multiclass image classification article](https://towardsdatascience.com/end-to-end-pipeline-for-setting-up-multiclass-image-classification-for-data-scientists-2e051081d41c)

[Also](https://medium.com/swlh/convolutional-neural-networks-for-multiclass-image-classification-a-beginners-guide-to-6dbc09fabbd)

## <font color = "lightblue"> Mount drive </font>
*You may need to visit CSC466 folder, and 'Add Shortcut to Drive' or 'Move To > My Drive"*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
datadir = "drive/MyDrive/CSC466/csc466_data/"

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/CSC466/
!cd drive/MyDrive/CSC466/csc466_data/

 csc466_data
'CSC 466 Project.ipynb'
'CSC 466 Team 13 Project Pitch.gdoc'
 food.zip
'KDD Project Proposal - Warmup Group 13.gdoc'


# <font color = "lightblue"> Build text files for X,y </font>
 - This is to build the dataset more quickly 
 - <font color="lime">These files are now built and in the csc466_data folder</font>
 - Running this block is not necessary currently, unless you want to shuffle the data again


In [ ]:
import os
import random

labels = os.listdir(datadir+"/images_train")
X = []
y = []

#read directories
for label in labels:
  food_dir = datadir +"images/"+ label
  food_images = os.listdir(food_dir)
  X.extend(food_images) 
  y.extend([label]*len(food_images))

#shuffle the data
data = list(zip(X, y))
random.shuffle(data)
X, y = zip(*data)

#write out to /CSC466/466_data/
with open(datadir+"X.txt","w") as Xfile:
  for count,filename in enumerate(X):
    Xfile.write(f"{y[count]}/{filename}\n")
  print(f"X.txt -- Complete: {len(X)} items")

with open(datadir+"y.txt","w") as Xfile:
  for filename in y:
    Xfile.write(f"{filename}\n")
  print(f"y.txt -- Complete: {len(y)} items")

X.txt -- Complete: 101016 items
y.txt -- Complete: 101016 items


## <font color = "lightblue">  Load X,y from text files </font>

In [ ]:
import os
import math
import random

X = []
y = []
X_train = []
y_train = []
X_test = []
y_test = []

#read
with open(datadir+"X.txt","r") as Xfile:
  X = Xfile.readlines()
with open(datadir+"y.txt","r") as yfile:
  y = yfile.readlines()

#split
split = math.ceil(len(X)*0.80)
X_train = X[:split]
X_test  = X[split:]
y_train = y[:split]
y_test  = y[split:]

print(f"Train len: {len(X_train)} | Test len: {len(X_test)}")

Train len: 80813 | Test len: 20203


# <font color = "lightblue"> Process images </font>

In [ ]:
# import statements
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# loading training data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        datadir+"images_train",
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# loading testing data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = train_datagen.flow_from_directory(
        datadir+"images_test",
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

Found 90923 images belonging to 101 classes.
Found 7539 images belonging to 101 classes.


# <font color = "lightblue"> Build model </font>

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=48, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(128, activation='relu'))
cnn.add(tf.keras.layers.Dense(101, activation='softmax'))

# finally compile and train the cnn
cnn.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
cnn.fit(x=train_generator, validation_data=test_generator, batch_size=32, epochs=30, verbose=True)
print(cnn.summary())

Epoch 1/30
   8/2842 [..............................] - ETA: 6:29:05 - loss: 4.6217 - accuracy: 0.0109

KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras import layers

lr = 3e-4
batch_size = 32
epochs = 200

model = tf.keras.Sequential(
    [
     layers.Conv2D(32,3,activation='relu',padding='same',name='conv1a', input_shape=[64,64,3]),
     layers.Conv2D(32,3,activation='relu',padding='same',name='conv1b'),
     layers.MaxPooling2D(2,2),
     layers.Conv2D(64,3,activation='relu',padding='same',name='conv2a'),
     layers.Conv2D(64,3,activation='relu',padding='same',name='conv2b'),
     layers.MaxPooling2D(2,2),
     layers.Conv2D(128,3,activation='relu',padding='same',name='conv3a'),
     layers.Conv2D(128,3,activation='relu',padding='same',name='conv3b'),
     layers.MaxPooling2D(2,2),
     layers.Flatten(),
     layers.Dense(1024,activation='relu',name='dense1'),
     layers.Dense(21,activation='softmax',name='z')
    ]
)
opt = tf.keras.optimizers.Adam(lr=lr)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics='accuracy')
history = model.fit(x=train_generator,validation_data=test_generator,batch_size=batch_size,epochs=epochs,verbose=True)



Epoch 1/200


InvalidArgumentError: ignored

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('iteration')
plt.ylabel('loss')
plt.title('Loss over time')
plt.legend(['train','val'])
plt.show()

# <font color = "lightblue"> Evaluate model </font>